# FOOD WASTE MANAGEMENT

## SQL DATABASE

In [2]:
import sqlite3
import pandas as pd

# Connect to SQLite 
conn = sqlite3.connect("lfwms.db")
cursor = conn.cursor()

# === Step 1: Create Tables ===
cursor.executescript("""
DROP TABLE IF EXISTS claims;
DROP TABLE IF EXISTS food_listings;
DROP TABLE IF EXISTS receivers;
DROP TABLE IF EXISTS providers;

CREATE TABLE providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
);

CREATE TABLE receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
);

CREATE TABLE food_listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER CHECK (Quantity > 0),
    Expiry_Date DATE,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID)
);

CREATE TABLE claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT CHECK (Status IN ('Pending','Completed','Cancelled')),
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID)
);
""")
conn.commit()

print("✅ Tables created successfully.")

# === Step 2: Load Clean Data into Tables ===
providers = pd.read_csv("clean/providers.csv")
receivers = pd.read_csv("clean/receivers.csv")
food_listings = pd.read_csv("clean/food_listings.csv")
claims = pd.read_csv("clean/claims.csv")

providers.to_sql("providers", conn, if_exists="append", index=False)
receivers.to_sql("receivers", conn, if_exists="append", index=False)
food_listings.to_sql("food_listings", conn, if_exists="append", index=False)
claims.to_sql("claims", conn, if_exists="append", index=False)

print("✅ Data loaded into database.")

# === Step 3: Row Counts Check ===
for table in ["providers", "receivers", "food_listings", "claims"]:
    count = cursor.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count} rows")

# Keeping connection open for next CRUD demo


✅ Tables created successfully.
✅ Data loaded into database.
providers: 1000 rows
receivers: 1000 rows
food_listings: 1000 rows
claims: 1000 rows


## CRUD OPERATIONS DEMO

In [3]:
# === Step 4: CRUD Operations Demo ===

print("\n=== CRUD Operations ===")

# CREATE: Insert a new food listing
cursor.execute("""
INSERT INTO food_listings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (2001, "Rice Packets", 50, "2025-08-30", 1, "Supermarket", "Mumbai", "Veg", "Dinner"))
conn.commit()
print("CREATE ✅ Added new food listing.")

# READ: Fetch food listings in Mumbai
rows = cursor.execute("SELECT Food_ID, Food_Name, Quantity, Location FROM food_listings WHERE Location='Mumbai' LIMIT 5").fetchall()
print("READ ✅ Sample listings in Mumbai:")
for row in rows:
    print(row)

# UPDATE: Mark a claim as Completed
cursor.execute("UPDATE claims SET Status='Completed' WHERE Claim_ID=1")
conn.commit()
print("UPDATE ✅ Claim_ID=1 marked as Completed.")

# DELETE: Remove a cancelled listing
cursor.execute("DELETE FROM food_listings WHERE Food_ID=2001")
conn.commit()
print("DELETE ✅ Removed the test food listing (Food_ID=2001).")



=== CRUD Operations ===
CREATE ✅ Added new food listing.
READ ✅ Sample listings in Mumbai:
(2001, 'Rice Packets', 50, 'Mumbai')
UPDATE ✅ Claim_ID=1 marked as Completed.
DELETE ✅ Removed the test food listing (Food_ID=2001).


In [4]:
conn.commit()   # save changes
conn.close()    # close properly


In [5]:
conn = sqlite3.connect("lfwms.db")
